In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

Date Alignment


In [ ]:
df_trans = pd.read_csv('C:\\Users\\Aman\\Desktop\\kifyaw1\\yfinance_data\\transformed_raw.csv')
df_apple = pd.read_csv('C:\\Users\\Aman\Desktop\kifyaw1\\yfinance_data\\AAPL_historical_data.csv')


df_apple['Date'] = pd.to_datetime(df_apple['Date'],format='mixed')
df_trans['Date'] = pd.to_datetime(df_trans['date'], format='mixed')
df_trans.drop('date', axis=1)

print(df_trans.dtypes)
print(df_apple.dtypes)

# df_merged = pd.merge(df_trans, df_apple, on='Date', how=' inner')


# print(df_merged)


<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Aman\AppData\Local\Temp\ipykernel_2060\3420039005.py:1: SyntaxWarning: invalid escape sequence '\A'
  df_trans = pd.read_csv('C:\\Users\Aman\\Desktop\\kifyaw1\\yfinance_data\\transformed_raw.csv')
C:\Users\Aman\AppData\Local\Temp\ipykernel_2060\3420039005.py:2: SyntaxWarning: invalid escape sequence '\D'
  df_apple = pd.read_csv('C:\\Users\\Aman\Desktop\kifyaw1\\yfinance_data\\AAPL_historical_data.csv')
C:\Users\Aman\AppData\Local\Temp\ipykernel_2060\3420039005.py:6: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

Unnamed: 0           int64
headline            object
url                 object
publisher           object
date                object
stock               object
datetime            object
sentiment_score    float64
sentiment           object
tokens              object
topic                int64
Date                object
dtype: object
Date            datetime64[ns]
Open                   float64
High                   float64
Low                    float64
Close                  float64
Adj Close              float64
Volume                   int64
Dividends              float64
Stock Splits           float64
dtype: object


In [15]:
df_trans.head()

,Unnamed: 0,headline,url,publisher,date,stock,datetime,sentiment_score,sentiment,tokens,topic,Date
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 14:30:54+00:00,0.000,Neutral,"['Stocks', 'That', 'Hit', 'Week', 'Highs', 'On...",2,2020-06-05 10:30:54-04:00
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,2020-06-03 14:45:20+00:00,0.000,Neutral,"['Stocks', 'That', 'Hit', 'Week', 'Highs', 'On...",2,2020-06-03 10:45:20-04:00
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,2020-05-26 08:30:07+00:00,0.000,Neutral,"['Biggest', 'Movers', 'From', 'Friday']",4,2020-05-26 04:30:07-04:00
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,2020-05-22 16:45:06+00:00,0.000,Neutral,"['Stocks', 'Moving', 'In', 'Fridays', 'MidDay'...",2,2020-05-22 12:45:06-04:00
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,2020-05-22 15:38:59+00:00,0.296,Positive,"['B', 'A', 'Securities', 'Maintains', 'Neutral...",3,2020-05-22 11:38:59-04:00
